# Soulection Spotify Playlists
This Jupyter Notebook has been created to code and test a tool that automatically creates a Spotify
 playlist from a Soulection Radio show using the Soulection Radio show ID.

In [1]:
# Import modules
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import spotipy
import spotipy.util as util
import base64
import re

In [19]:
# Function to return ids of Spotify songs
USER_AGENT = 'User-Agent'
MOZILLA_GENERAL_TOKEN = 'Mozilla/5.0'
HTML_SPOTIFY_TITLE = 'spotify'
HREF_ATTRIBUTE = 'href'
SPLITTER = 'track/'

URL = 'https://soulectiontracklists.com/episodes/'
HEADER = {USER_AGENT:MOZILLA_GENERAL_TOKEN}

SPOTIFY_HREFS_LIST = list()
SPOTIFY_IDS_LIST = list()
CLEANED_SPOTIFY_IDS_LIST = list()

def loop_spotify_titles(spotify_id_list, spotify_title_soup):
    """

    :param spotify_id_list:
    :param spotify_title_soup:
    :return:
    """
    for spotify_title in spotify_title_soup:
        spotify_id_list.append(spotify_title[HREF_ATTRIBUTE])

    return spotify_id_list

def loop_spotify_ids(spotify_ids_list, spotify_href_list):
    """

    :param spotify_ids_list:
    :param spotify_href_list:
    :return:
    """
    for spotify_song_id in spotify_href_list:
        spotify_ids_list.append(spotify_song_id.split(SPLITTER)[1])

    return spotify_ids_list


def clean_spotify_ids(clean_spotify_ids_list, spotify_ids_list):
    """

    :param clean_spotify_ids_list:
    :param spotify_ids_list:
    :return:
    """
    for id in spotify_ids_list:
        clean_spotify_ids_list.append(id[:22])

    return clean_spotify_ids_list

def soulection_show_spotify_song_ids(show_id):
    """

    :param show_id:
    :return:
    """
    soulection_tracklist_url = URL + str(show_id)
    request = Request(soulection_tracklist_url, headers=HEADER)
    soulection_tracklist_url_response = urlopen(request)
    soup = BeautifulSoup(soulection_tracklist_url_response)
    spotify_title_soup = soup.find_all(title=HTML_SPOTIFY_TITLE)

    spotify_href_list = loop_spotify_titles(SPOTIFY_HREFS_LIST, spotify_title_soup)
    spotify_id_list = loop_spotify_ids(SPOTIFY_IDS_LIST, spotify_href_list)
    clean_spotify_ids_list = clean_spotify_ids(CLEANED_SPOTIFY_IDS_LIST, spotify_id_list)

    return clean_spotify_ids_list

In [20]:
SOULECTION_SHOW_NUMBER = 462
# Return ids of Spotify songs on Soulection show 481
show_spotify_ids = soulection_show_spotify_song_ids(SOULECTION_SHOW_NUMBER)
show_spotify_ids

['06bjoUJjesDCfM1vjajvZQ',
 '3ppVO2tyWRRznNmONvt7Se',
 '4hGEJ3VhpJJxxEqQnz9JPS',
 '6q0K18setJn0aOVR99Us0p',
 '0ritGnnjhOz4jBiUUIJyHR',
 '4GoKFTTm5DfBhzSCd0BLMX',
 '5Xt3RUiOkiH2pGYtMTVeNS',
 '25z0QF3ANJPVM04fTbty45',
 '2Q7XXvTB9EQDEAjRvvORMF',
 '07qj9w93jk1eA2TGAd6kxj',
 '4qh6Aw7Y7G4rxtRB3SUmqs',
 '1pmVrxB7KlxsPfikuq8i0M',
 '4QJQJimv59kekwAk2umyUS',
 '1r0IUucGcijDB48bXk4Z7p',
 '1mcfqQYSMsbDzmg9AvynoC',
 '1mcfqQYSMsbDzmg9AvynoC',
 '2mTyh3nBpLJZBTb3M6nVzP',
 '01wKeKPgYxdPYA8rfMveYc',
 '39tE9pvzHERJaFJ3HviIPj',
 '1KwIHDiA7GqySNMrTH0A8f',
 '5LdrV2BHroDGfgJFEZVZki',
 '4vmpBY1GrdNcS3ugNHf3ay',
 '7gPQXKzOY7EBBheoW0xQC4',
 '7gPQXKzOY7EBBheoW0xQC4',
 '42Oij6tTNlS0zy5HfNZWva',
 '13sjXB4DOFvBe2xWoQqHxK',
 '33wUd3GgaRf2rux2aJ71yb',
 '3TiCuJiisBOB9HzKAM6FDG',
 '4YmKTCL86mpGHUS0sEWAUV',
 '515MVhJ5mWAk41VfyOrezB',
 '7aoslYDlH3JDogn6uA1kIo',
 '7l6Nxj3XhF4iUV5Fy0OxmA',
 '0IRK3U73ve4mmGGySqdNY4',
 '2hFHoGxlgYtjnsY34kTyHo',
 '5LOIteqUpPswpOEVQcB6NC',
 '4CIyRJ42G522wF7WzJqPF7',
 '0Kj9GWYI2CzSnkmjFmI1Yd',
 

In [4]:
# Function for Spotipy authentication
SPOTIFY_API_CREDENTIALS = pd.read_csv('spotify_api_keys.csv', header=None)

SPOTIFY_USER_ID = SPOTIFY_API_CREDENTIALS[1].iloc[0]
SPOTIFY_CLIENT_ID = SPOTIFY_API_CREDENTIALS[1].iloc[1]
SPOTIFY_CLIENT_SECRET = SPOTIFY_API_CREDENTIALS[1].iloc[2]
REDIRECT_URL = 'http://127.0.0.1:9090'

## All scope
SPOTIFY_AUTHORIZATION_SCOPE = 'ugc-image-upload user-read-playback-state streaming ' \
                              'user-read-email playlist-read-collaborative '  \
                              'user-modify-playback-state user-read-private  ' \
                              'playlist-modify-public user-library-modify user-top-read ' \
                              'user-read-playback-position user-read-currently-playing ' \
                              'playlist-read-private user-follow-read app-remote-control ' \
                              'user-read-recently-played user-follow-modify user-library-read'

def spotify_authentication(spotify_user_id, spotify_authorization_scope, spotify_client_id,
                           spotify_client_secret, redicrect_url):
    """

    :param spotify_user_id:
    :param spotify_authorization_scope:
    :param spotify_client_id:
    :param spotify_client_secret:
    :param redicrect_url:
    :return:
    """
    token = util.prompt_for_user_token(
        spotify_user_id,
        spotify_authorization_scope,
        client_id = spotify_client_id,
        client_secret = spotify_client_secret,
        redirect_uri=redicrect_url)

    sp = spotipy.Spotify(auth=token)

    return sp

In [5]:
# Spotipy authentication
sp = spotify_authentication(SPOTIFY_USER_ID, SPOTIFY_AUTHORIZATION_SCOPE, SPOTIFY_CLIENT_ID,
                       SPOTIFY_CLIENT_SECRET, REDIRECT_URL)


In [6]:
PLAYLIST_PREFFIX = 'Soulection Show'
PLAYLIST_NAME = PLAYLIST_PREFFIX + ' ' + str(SOULECTION_SHOW_NUMBER)

def create_and_add_tracks_to_playlist(spotify_song_ids, playlist_name):
    """

    :param spotify_song_ids:
    :param playlist_name:
    :return:
    """
    playlist = sp.user_playlist_create(SPOTIFY_USER_ID, playlist_name)
    sp.playlist_add_items(playlist['id'], spotify_song_ids)

    print(f'Spotify playlist "{playlist_name}" successfully created!')

    return playlist['id']

In [7]:
created_playlist_id = create_and_add_tracks_to_playlist(show_spotify_ids, PLAYLIST_NAME)

Spotify playlist "Soulection Show 461" successfully created!


In [8]:
# Deletes playlist 'Pythonically Created' using Playlist Id
#sp.current_user_unfollow_playlist('2x3WGyPocjz53pRd0Q4vwT')

## Download Show Art and Upload as Spotify Cover Art

In [6]:
ALT_TAG = 'Show #'
SRCSET_ATTRIBUTE = 'srcset'
SPLIT_BY_COMMA = ','
SPLIT_BY_SPACE = ' '
INDEX_2 = 2
INDEX_1 = 1
INDEX_0 = 0
REGEX_WILDCARD = '.*'

# Function that returns the href of the cover art
def soulection_show_art_cover_href(show_id, index):
    """

    :param show_id:
    :param index:
    :return:
    """
    soulection_tracklist_url = URL + str(show_id)
    request = Request(soulection_tracklist_url, headers=HEADER)
    soulection_tracklist_url_response = urlopen(request)
    soup = BeautifulSoup(soulection_tracklist_url_response)
    soulection_show_cover_art_soup = soup.find(alt=re.compile(ALT_TAG + str(show_id) +
                                                              REGEX_WILDCARD))

    soulection_show_cover_art_href_string = soulection_show_cover_art_soup[SRCSET_ATTRIBUTE].split \
        (SPLIT_BY_COMMA)[index].split(SPLIT_BY_SPACE)[INDEX_0][1:]

    return soulection_show_cover_art_href_string


IMAGE_FOLDER = r'C:\Users\Jaume\Documents\Python Projects\soulection_playlists\''
JPEG_STRING = '.jpeg'
WRITING_BINARY = 'wb'
READ_BINARY = 'rb'
UTF_8_ENCODING = 'utf-8'

# Function that saves the cover art href to a jpeg on the local
def save_jepg_from_href(href):
    """

    :param href:
    :return:
    """
    file = open(IMAGE_FOLDER + PLAYLIST_NAME + JPEG_STRING, WRITING_BINARY)
    file.write(requests.get(href).content)
    file.close()

    pass

# Function that encodes the jpeg to base64 (required by Spotify)
def get_base64_encoded_image(image_path):
    """

    :param image_path:
    :return:
    """
    with open(image_path, READ_BINARY) as img_file:
        return base64.b64encode(img_file.read()).decode(UTF_8_ENCODING)


# Function that groups the entire cover art process
def upload_spotify_playlist_cover_art(show_ids_list, index, playlist_id):
    """

    :param show_ids_list:
    :param index:
    :param playlist_id:
    :return:
    """

    href = soulection_show_art_cover_href(show_ids_list, index)
    save_jepg_from_href(href)
    base64_encoded_cover_art = get_base64_encoded_image(IMAGE_FOLDER + PLAYLIST_NAME +
                                                        JPEG_STRING)
    sp.playlist_upload_cover_image(playlist_id, base64_encoded_cover_art)

    pass

In [10]:
# Tries to upload the crispest best quality image first and if fails it attempts a lower res.
try:
    upload_spotify_playlist_cover_art(SOULECTION_SHOW_NUMBER, INDEX_1, created_playlist_id)
except:
    upload_spotify_playlist_cover_art(SOULECTION_SHOW_NUMBER, INDEX_2, created_playlist_id)


In [14]:
# Make playlist public and change description
PLAYLIST_DESCRIPTION = f'This playlist contains songs, in order, from Soulection Radio Show ' \
                       f'{SOULECTION_SHOW_NUMBER}. The playlist only has songs that can be found ' \
                       f'on Spotify and therefore is missing "SoundCloud gems". You are welcome ' \
                       f'to share it with friends! This playlist and all of the others ' \
                       f'(Soulection) have been created automatically using Python.'
sp.playlist_change_details(created_playlist_id, public=True, description=PLAYLIST_DESCRIPTION)

'Soulection Show 460'

In [2]:
SOULECTION_SHOW_ID = 462

# Function to return ids of Spotify songs
USER_AGENT = 'User-Agent'
MOZILLA_GENERAL_TOKEN = 'Mozilla/5.0'
HTML_SPOTIFY_TITLE = 'spotify'
HREF_ATTRIBUTE = 'href'
SPLITTER = 'track/'

URL = 'https://soulectiontracklists.com/episodes/'
HEADER = {USER_AGENT:MOZILLA_GENERAL_TOKEN}

SPOTIFY_HREFS_LIST = list()
SPOTIFY_IDS_LIST = list()
CLEANED_SPOTIFY_IDS_LIST = list()

def loop_spotify_titles(spotify_id_list, spotify_title_soup):
    """

    :param spotify_id_list:
    :param spotify_title_soup:
    :return:
    """
    for spotify_title in spotify_title_soup:
        spotify_id_list.append(spotify_title[HREF_ATTRIBUTE])

    return spotify_id_list

def loop_spotify_ids(spotify_ids_list, spotify_href_list):
    """

    :param spotify_ids_list:
    :param spotify_href_list:
    :return:
    """
    for spotify_song_id in spotify_href_list:
        spotify_ids_list.append(spotify_song_id.split(SPLITTER)[1])

    return spotify_ids_list


def clean_spotify_ids(clean_spotify_ids_list, spotify_ids_list):
    """

    :param clean_spotify_ids_list:
    :param spotify_ids_list:
    :return:
    """
    for id in spotify_ids_list:
        clean_spotify_ids_list.append(id[:22])

    return clean_spotify_ids_list

def soulection_show_spotify_song_ids(show_id):
    """

    :param show_id:
    :return:
    """
    soulection_tracklist_url = URL + str(show_id)
    request = Request(soulection_tracklist_url, headers=HEADER)
    soulection_tracklist_url_response = urlopen(request)
    soup = BeautifulSoup(soulection_tracklist_url_response)
    spotify_title_soup = soup.find_all(title=HTML_SPOTIFY_TITLE)

    spotify_href_list = loop_spotify_titles(SPOTIFY_HREFS_LIST, spotify_title_soup)
    spotify_id_list = loop_spotify_ids(SPOTIFY_IDS_LIST, spotify_href_list)
    clean_spotify_ids_list = clean_spotify_ids(CLEANED_SPOTIFY_IDS_LIST, spotify_id_list)

    return clean_spotify_ids_list


# Function for Spotipy authentication
SPOTIFY_API_CREDENTIALS = pd.read_csv(r'C:\Users\Jaume\Documents\Python '
                                      r'Projects\soulection_playlists\spotify_api_keys.csv',
                                      header=None)

SPOTIFY_USER_ID = SPOTIFY_API_CREDENTIALS[1].iloc[0]
SPOTIFY_CLIENT_ID = SPOTIFY_API_CREDENTIALS[1].iloc[1]
SPOTIFY_CLIENT_SECRET = SPOTIFY_API_CREDENTIALS[1].iloc[2]
REDIRECT_URL = 'http://127.0.0.1:9090'

## All scope
SPOTIFY_AUTHORIZATION_SCOPE = 'ugc-image-upload user-read-playback-state streaming ' \
                              'user-read-email playlist-read-collaborative '  \
                              'user-modify-playback-state user-read-private  ' \
                              'playlist-modify-public user-library-modify user-top-read ' \
                              'user-read-playback-position user-read-currently-playing ' \
                              'playlist-read-private user-follow-read app-remote-control ' \
                              'user-read-recently-played user-follow-modify user-library-read'

def spotify_authentication(spotify_user_id, spotify_authorization_scope, spotify_client_id,
                           spotify_client_secret, redicrect_url):
    """

    :param spotify_user_id:
    :param spotify_authorization_scope:
    :param spotify_client_id:
    :param spotify_client_secret:
    :param redicrect_url:
    :return:
    """
    token = util.prompt_for_user_token(
        spotify_user_id,
        spotify_authorization_scope,
        client_id = spotify_client_id,
        client_secret = spotify_client_secret,
        redirect_uri=redicrect_url)

    sp = spotipy.Spotify(auth=token)

    return sp


sp = spotify_authentication(SPOTIFY_USER_ID, SPOTIFY_AUTHORIZATION_SCOPE, SPOTIFY_CLIENT_ID,
                       SPOTIFY_CLIENT_SECRET, REDIRECT_URL)


PLAYLIST_PREFFIX = 'Soulection Show'
PLAYLIST_NAME = PLAYLIST_PREFFIX + ' ' + str(SOULECTION_SHOW_ID)

def create_and_add_tracks_to_playlist(spotify_song_ids, playlist_name):
    """

    :param spotify_song_ids:
    :param playlist_name:
    :return:
    """
    playlist = sp.user_playlist_create(SPOTIFY_USER_ID, playlist_name)
    sp.playlist_add_items(playlist['id'], spotify_song_ids)

    return playlist['id']


ALT_TAG = 'Show #'
SRCSET_ATTRIBUTE = 'srcset'
SPLIT_BY_COMMA = ','
SPLIT_BY_SPACE = ' '
INDEX_2 = 2
INDEX_1 = 1
INDEX_0 = 0
REGEX_WILDCARD = '.*'

# Function that returns the href of the cover art
def soulection_show_art_cover_href(show_id, index):
    """

    :param show_id:
    :param index:
    :return:
    """
    soulection_tracklist_url = URL + str(show_id)
    request = Request(soulection_tracklist_url, headers=HEADER)
    soulection_tracklist_url_response = urlopen(request)
    soup = BeautifulSoup(soulection_tracklist_url_response)
    soulection_show_cover_art_soup = soup.find(alt=re.compile(ALT_TAG + str(show_id) +
                                                              REGEX_WILDCARD))

    soulection_show_cover_art_href_string = soulection_show_cover_art_soup[SRCSET_ATTRIBUTE].split \
        (SPLIT_BY_COMMA)[index].split(SPLIT_BY_SPACE)[INDEX_0][1:]

    return soulection_show_cover_art_href_string


IMAGE_FOLDER = r'C:\Users\Jaume\Documents\Python Projects\soulection_playlists\''
JPEG_STRING = '.jpeg'
WRITING_BINARY = 'wb'
READ_BINARY = 'rb'
UTF_8_ENCODING = 'utf-8'

# Function that saves the cover art href to a jpeg on the local
def save_jepg_from_href(href):
    """

    :param href:
    :return:
    """
    file = open(IMAGE_FOLDER + PLAYLIST_NAME + JPEG_STRING, WRITING_BINARY)
    file.write(requests.get(href).content)
    file.close()

    pass

# Function that encodes the jpeg to base64 (required by Spotify)
def get_base64_encoded_image(image_path):
    """

    :param image_path:
    :return:
    """
    with open(image_path, READ_BINARY) as img_file:
        return base64.b64encode(img_file.read()).decode(UTF_8_ENCODING)


# Function that groups the entire cover art process
def upload_spotify_playlist_cover_art(show_ids_list, index, playlist_id):
    """

    :param show_ids_list:
    :param index:
    :param playlist_id:
    :return:
    """

    href = soulection_show_art_cover_href(show_ids_list, index)
    save_jepg_from_href(href)
    base64_encoded_cover_art = get_base64_encoded_image(IMAGE_FOLDER + PLAYLIST_NAME +
                                                        JPEG_STRING)
    sp.playlist_upload_cover_image(playlist_id, base64_encoded_cover_art)

    pass


PLAYLIST_DESCRIPTION = f'This playlist contains songs, in order, from Soulection Radio Show ' \
                       f'{SOULECTION_SHOW_ID}. The playlist only has songs that can be found ' \
                       f'on Spotify and therefore is missing "SoundCloud gems". You are welcome ' \
                       f'to share it with friends! This playlist and all of the others ' \
                       f'(Soulection) have been created automatically using Python.'


def soulection_show_to_spotify_playlist(soulection_show_id):

    show_spotify_ids = soulection_show_spotify_song_ids(soulection_show_id)
    created_playlist_id = create_and_add_tracks_to_playlist(show_spotify_ids, PLAYLIST_NAME)

    try:
        upload_spotify_playlist_cover_art(soulection_show_id, INDEX_1, created_playlist_id)
    except:
        upload_spotify_playlist_cover_art(soulection_show_id, INDEX_2, created_playlist_id)

    sp.playlist_change_details(created_playlist_id, public=True, description=PLAYLIST_DESCRIPTION)

    pass

soulection_show_to_spotify_playlist(SOULECTION_SHOW_ID)

Spotify playlist "Soulection Show 461" successfully created!
